In [1]:
import pandas as pd
data = pd.read_csv("../doc2vec/data/smalldf-1000notebooks.csv")

In [ ]:
len(data[data.cell_type=="code"])

In [2]:
import numpy as np
embeds2d = np.load("../doc2vec/data/notebooks-sliced-doc2vec-vectors-apr7-small.npy",allow_pickle=True)


In [3]:
embeds1d = []
for row in embeds2d:
    for vecs in row:
        embeds1d.append(vecs)

In [4]:
len(embeds1d)

22303

In [ ]:
import argparse
import logging
import os
import pickle
import random
import torch
import json
import numpy as np
import pandas as pd
from model import BertModel, Generator
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from gensim.models.doc2vec import Doc2Vec
import math
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange
from pathlib import Path
import codecs
import json

from utils import CellFeatures, InputFeatures, convert_examples_to_features, parseNotebook, get_notebook_list, get_embedding
from config import *

class RetrivalDB:
  def __init__(self):
    doc2vec_path = '../doc2vec/model'
    self.embed = np.asarray(embeds1d)
    self.df = pd.read_csv("../doc2vec/data/smalldf-1000notebooks.csv")
    self.df = self.df[self.df.cell_type == 'code']
    # self.kernel_ids = np.load(doc2vec_path + "codebase_doc2vec_id.npy", allow_pickle=True)

  def getDoc(self, raw_idx):
    print("Raw idx is", raw_idx)
    if raw_idx < 0 or raw_idx >= self.embed.shape[0]:
      print("ERROR: out of index")
      return None
    
    return self.df.iloc[raw_idx]

  def find_sim(self, embed, topn=10):
    result = np.einsum("ij,ij->i",self.embed,embed)
    rank = np.argsort(-result)[:topn]
    doc_list = [self.getDoc(r) for r in rank]
    return doc_list

if __name__ == "__main__":
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # TODO: change to doc2vec
  doc2vec_path = '../doc2vec/model'
  model = Doc2Vec.load(doc2vec_path + "/notebook-doc2vec-model-apr7-1000notebooks.model")

  # gen = torch.load(doc2vec_path + "/best_gen.pt").to(device)
  gen = Generator(768, 768).to(device)
  gen = torch.load('gen_saved/best_gen.pt')
  gen.eval()
  db = RetrivalDB()
  
  while(True):
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    input("Update the sample.py and press Enter to continue...")
    # TODO: reads ipynb
    input_file = './sample.ipynb'
    embed_list = []
    f = codecs.open(input_file, 'r')
    source = f.read()

    y = json.loads(source)
    for x in y['cells']:
        for x2 in x['source']:
            if x2[-1] != '\n':
                x2 = x2 + '\n'
            print("Input is", x2)
            embed_list.append(torch.Tensor(model.infer_vector(x2.split(' '))).to(device))

    print([e.shape for e in embed_list])

    predict_embed = gen.generate_embedding(embed_list)

    predict_embed = [embed.detach().cpu().numpy() for embed in predict_embed]

    doc_list = db.find_sim(predict_embed, topn=3)

    file_path = '../doc2vec/data/sliced-notebooks-full-new'

    for col in doc_list:
#       print(col)
      print("############################")
      print(col.filename, col.competition)
      print(col.source)


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


Update the sample.py and press Enter to continue... 


Input is import pandas as pd

Input is df = pd.read_csv('1.csv')

Input is df.head()

[torch.Size([768]), torch.Size([768]), torch.Size([768])]
Raw idx is 19602
Raw idx is 19016
Raw idx is 20519
############################
35082405 tweet-sentiment-extraction
## now do the same for selected text
 selected_shap_extracted = []
 for i in np.arange(len(df_train)):
     selected_shap_extracted.append(shap_values[i, df_train['selected_text_word_index'][i]])
############################
6186489 ga-customer-revenue-prediction
# Extract target values and Ids
 cat_cols = ['channelGrouping','device.browser',
        'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
        'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
        'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.
############################
7237023 pubg-finish-placement-prediction
corr = train_df[['walkDistance', 'players_in_team','total_kickass_score', 'winPlacePerc']].corr()
 sns.he

In [4]:
gen = torch.load('gen_saved/best_gen.pt')
gen.eval()

Generator(
  (gru_unit): GRU(768, 768)
  (trans): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [4]:
model = torch.load("gen_saved/doc2vec_best_gen.pt")